## 🧾 Etapa 5.1 — Leitura e Inspeção Inicial dos Dados Brutos (Atualizada)

Após reestruturação do ambiente, os arquivos de dados foram organizados no diretório `/workspace/dados`, acessível diretamente pelo Jupyter Notebook. 

O objetivo deste bloco permanece: listar os arquivos `.csv` disponíveis e verificar sua acessibilidade antes de prosseguir para o carregamento e inspeção detalhada.


In [5]:
import os

# Caminho relativo real identificado
caminho_dados = "../dados"

# Listar arquivos .csv disponíveis
arquivos_csv = [f for f in os.listdir(caminho_dados) if f.endswith('.csv')]

# Exibir a lista
print("📂 Arquivos CSV encontrados em ../dados:")
for nome_arquivo in arquivos_csv:
    print(" -", nome_arquivo)


📂 Arquivos CSV encontrados em ../dados:
 - dados_bova11.csv
 - dados_petr4.csv
 - dados_brfs3.csv
 - dados_imab11.csv
 - dados_vale3.csv


## 🧾 Etapa 5.1 — Leitura Focada nos Ativos do Agente RL

Com base no papel dos arquivos disponíveis, nesta etapa focaremos apenas nos dados históricos dos ativos que farão parte da política de decisão do agente: **VALE3**, **PETR4** e **BRFS3**.

Os dados de `BOVA11` e `IMAB11` serão mantidos intactos por enquanto, e utilizados mais adiante como benchmarks para comparação de performance.


In [6]:
import pandas as pd

# Caminho onde estão os arquivos
caminho_dados = "../dados"

# Lista apenas os arquivos que nos interessam nesta etapa
arquivos_relevantes = ["dados_vale3.csv", "dados_petr4.csv", "dados_brfs3.csv"]

# Carregamento em dicionário de DataFrames
ativos_rl = {}

for arquivo in arquivos_relevantes:
    nome_ativo = arquivo.replace("dados_", "").replace(".csv", "").upper()
    caminho_completo = os.path.join(caminho_dados, arquivo)

    df = pd.read_csv(caminho_completo)
    ativos_rl[nome_ativo] = df

    print(f"\n📊 {nome_ativo} — Primeiras linhas:")
    display(df.head())
    print("\nTipos de dados:")
    print(df.dtypes)
    print("\nValores nulos por coluna:")
    print(df.isnull().sum())



📊 VALE3 — Primeiras linhas:


,Price,Close,High,Low,Open,Volume
0,Ticker,VALE3.SA,VALE3.SA,VALE3.SA,VALE3.SA,VALE3.SA
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-05-06,27.61468505859375,27.882544683461767,27.166171498244843,27.527472240287658,26602700
3,2020-05-07,28.686128616333008,29.26545779786383,28.312368927225574,28.42449730921903,42194300
4,2020-05-08,30.43034553527832,30.498868701842245,29.259230533971,29.58938501689946,33922800



Tipos de dados:
Price     object
Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object

Valores nulos por coluna:
Price     0
Close     1
High      1
Low       1
Open      1
Volume    1
dtype: int64

📊 PETR4 — Primeiras linhas:


,Price,Close,High,Low,Open,Volume
0,Ticker,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-05-06,6.878803253173828,7.18930429063302,6.878803253173829,7.113669850843073,67937500
3,2020-05-07,6.942496299743652,7.11367034998188,6.906669137634476,7.065900547411228,81265100
4,2020-05-08,7.3564982414245605,7.396306285182488,7.049977291546384,7.061919932456346,83232700



Tipos de dados:
Price     object
Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object

Valores nulos por coluna:
Price     0
Close     1
High      1
Low       1
Open      1
Volume    1
dtype: int64

📊 BRFS3 — Primeiras linhas:


,Price,Close,High,Low,Open,Volume
0,Ticker,BRFS3.SA,BRFS3.SA,BRFS3.SA,BRFS3.SA,BRFS3.SA
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-05-06,16.95716094970703,17.318345287107338,16.670064327299208,17.077555110283047,9585305
3,2020-05-07,17.04977035522461,17.86475184327914,16.966419132277178,17.197947626715926,11953503
4,2020-05-08,17.355388641357422,17.61470026162958,16.73489271346732,17.58691713883929,8133059



Tipos de dados:
Price     object
Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object

Valores nulos por coluna:
Price     0
Close     1
High      1
Low       1
Open      1
Volume    1
dtype: int64


## 🔧 Correção da Leitura: Skip de Cabeçalho e Nomeação das Colunas

Foi identificado que os arquivos possuem três linhas superiores com metadados (nomes de ativo e rótulos), o que desalinha os dados ao fazer a leitura padrão.

Esta célula corrige o problema utilizando `skiprows=3` e definindo manualmente os nomes das colunas para garantir consistência na leitura dos arquivos dos ativos VALE3, PETR4 e BRFS3.


In [9]:
import pandas as pd
import os

# Caminho correto
caminho_dados = "../dados"

# Arquivos a serem lidos
arquivos_relevantes = ["dados_vale3.csv", "dados_petr4.csv", "dados_brfs3.csv"]

# Colunas padronizadas
colunas_padrao = ["Date", "Close", "High", "Low", "Open", "Volume"]

# Leitura limpa
ativos_rl = {}

for arquivo in arquivos_relevantes:
    nome_ativo = arquivo.replace("dados_", "").replace(".csv", "").upper()
    caminho_completo = os.path.join(caminho_dados, arquivo)

    df = pd.read_csv(caminho_completo, skiprows=3, names=colunas_padrao)
    ativos_rl[nome_ativo] = df

    print(f"\n📊 {nome_ativo} — Primeiras linhas:")
    display(df.head())



📊 VALE3 — Primeiras linhas:


,Date,Close,High,Low,Open,Volume
0,2020-05-06,27.614685,27.882545,27.166171,27.527472,26602700
1,2020-05-07,28.686129,29.265458,28.312369,28.424497,42194300
2,2020-05-08,30.430346,30.498869,29.259231,29.589385,33922800
3,2020-05-11,29.732660,30.679521,29.732660,30.031669,24783100
4,2020-05-12,29.564466,30.274611,29.564466,30.168714,21806900



📊 PETR4 — Primeiras linhas:


,Date,Close,High,Low,Open,Volume
0,2020-05-06,6.878803,7.189304,6.878803,7.113670,67937500
1,2020-05-07,6.942496,7.113670,6.906669,7.065901,81265100
2,2020-05-08,7.356498,7.396306,7.049977,7.061920,83232700
3,2020-05-11,7.225132,7.535634,7.209209,7.284844,65061500
4,2020-05-12,7.221150,7.495825,7.217170,7.376402,71837200



📊 BRFS3 — Primeiras linhas:


,Date,Close,High,Low,Open,Volume
0,2020-05-06,16.957161,17.318345,16.670064,17.077555,9585305
1,2020-05-07,17.049770,17.864752,16.966419,17.197948,11953503
2,2020-05-08,17.355389,17.614700,16.734893,17.586917,8133059
3,2020-05-11,19.309492,19.939250,18.170371,18.290765,30998471
4,2020-05-12,18.864956,19.874421,18.800128,19.522497,16563627


## 💾 Salvando os DataFrames Corrigidos

Após corrigirmos a leitura dos dados com `skiprows=3` e definirmos manualmente os nomes das colunas, salvaremos agora os arquivos limpos para uso futuro.

Os novos arquivos serão nomeados no formato:  
`<ativo>_clean.csv` → ex: `VALE3_clean.csv`, `PETR4_clean.csv`, `BRFS3_clean.csv`.

Eles serão gravados no mesmo diretório `../dados`.


In [10]:
# Salvar os DataFrames corrigidos no mesmo diretório
for nome_ativo, df in ativos_rl.items():
    nome_arquivo_saida = f"{nome_ativo}_clean.csv"
    caminho_saida = os.path.join(caminho_dados, nome_arquivo_saida)

    df.to_csv(caminho_saida, index=False)
    print(f"✅ {nome_arquivo_saida} salvo com sucesso em: {caminho_saida}")


✅ VALE3_clean.csv salvo com sucesso em: ../dados/VALE3_clean.csv
✅ PETR4_clean.csv salvo com sucesso em: ../dados/PETR4_clean.csv
✅ BRFS3_clean.csv salvo com sucesso em: ../dados/BRFS3_clean.csv


## 🗃️ Padronização de Tipos e Ordenação Temporal

Antes de calcular indicadores técnicos e construir o espaço de estados, é essencial garantir que os dados estejam com os tipos corretos e em ordem cronológica.

Nesta célula:
- Convertemos a coluna `Date` para tipo datetime.
- Ordenamos as linhas por data.
- Validamos os tipos numéricos.
- Sinalizamos valores ausentes (se houver).


In [11]:
# Padronização de tipos e ordenação por data
for nome_ativo, df in ativos_rl.items():
    print(f"\n📌 {nome_ativo}")

    # Conversão da coluna Date
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # Ordenar por data (ascendente)
    df.sort_values("Date", inplace=True)

    # Resetar índice após ordenação
    df.reset_index(drop=True, inplace=True)

    # Verificar tipos finais
    print("Tipos de dados após conversão:")
    print(df.dtypes)

    # Verificar nulos (caso algum erro de parsing gere datas faltantes)
    nulos = df.isnull().sum()
    if nulos.any():
        print("⚠️ Valores nulos encontrados:")
        print(nulos)
    else:
        print("✅ Nenhum valor nulo encontrado.")

    # Atualiza o dicionário
    ativos_rl[nome_ativo] = df



📌 VALE3
Tipos de dados após conversão:
Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
dtype: object
✅ Nenhum valor nulo encontrado.

📌 PETR4
Tipos de dados após conversão:
Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
dtype: object
✅ Nenhum valor nulo encontrado.

📌 BRFS3
Tipos de dados após conversão:
Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
dtype: object
✅ Nenhum valor nulo encontrado.


## ⚙️ Engenharia de Indicadores Técnicos (SMA, Volatilidade, RSI, Retornos)

A partir dos dados de fechamento diário, serão criados os seguintes indicadores:

- **Média Móvel Simples (SMA5 e SMA20)** para identificar tendências de curto e médio prazo;
- **Volatilidade (STD10)** como medida de risco;
- **Índice de Força Relativa (RSI14)** para detectar sobrecompra/sobrevenda;
- **Retorno Diário** para capturar o movimento percentual diário dos preços.

Esses indicadores serão adicionados como colunas aos DataFrames já limpos.


In [12]:
def calcular_indicadores(df):
    df = df.copy()

    # Retorno percentual diário
    df["Return"] = df["Close"].pct_change()

    # Médias móveis simples
    df["SMA5"] = df["Close"].rolling(window=5).mean()
    df["SMA20"] = df["Close"].rolling(window=20).mean()

    # Volatilidade (desvio padrão dos retornos)
    df["Volatility"] = df["Return"].rolling(window=10).std()

    # RSI (Índice de Força Relativa)
    delta = df["Close"].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))

    return df

# Aplica os indicadores a todos os ativos
for nome_ativo, df in ativos_rl.items():
    df_ind = calcular_indicadores(df)
    ativos_rl[nome_ativo] = df_ind

    print(f"\n✅ Indicadores calculados para {nome_ativo}")
    display(df_ind[["Date", "Close", "Return", "SMA5", "SMA20", "Volatility", "RSI"]].dropna().head())



✅ Indicadores calculados para VALE3


,Date,Close,Return,SMA5,SMA20,Volatility,RSI
19,2020-06-02,33.277157,0.000000,32.463605,31.035528,0.025094,67.994058
20,2020-06-03,33.426662,0.004493,32.839857,31.326126,0.025099,70.861386
21,2020-06-04,34.672531,0.037272,33.533806,31.625447,0.025087,74.853031
22,2020-06-05,34.018452,-0.018864,33.734392,31.804852,0.025326,62.737639
23,2020-06-08,34.124348,0.003113,33.903830,32.024436,0.025066,60.097731



✅ Indicadores calculados para PETR4


,Date,Close,Return,SMA5,SMA20,Volatility,RSI
19,2020-06-02,8.518887,0.052631,8.102497,7.502394,0.025243,80.286108
20,2020-06-03,8.546755,0.003271,8.225106,7.585791,0.024473,82.982160
21,2020-06-04,8.530830,-0.001863,8.357268,7.665208,0.024212,85.905978
22,2020-06-05,8.797543,0.031265,8.497392,7.737260,0.020689,83.969402
23,2020-06-08,8.968717,0.019457,8.672546,7.824440,0.018602,84.810054



✅ Indicadores calculados para BRFS3


,Date,Close,Return,SMA5,SMA20,Volatility,RSI
19,2020-06-02,21.624777,-0.013519,21.598847,20.183744,0.021110,54.578748
20,2020-06-03,20.976498,-0.029979,21.487714,20.384711,0.023150,46.554617
21,2020-06-04,21.717390,0.035320,21.532167,20.618091,0.022662,56.220471
22,2020-06-05,21.022804,-0.031983,21.452521,20.801462,0.025833,57.766997
23,2020-06-08,21.532166,0.024229,21.374727,20.912596,0.026475,59.659966


## ⚖️ Aplicação de Normalização Mista (por ativo)

Cada ativo terá seus dados normalizados individualmente, aplicando técnicas apropriadas para cada tipo de variável:

- **Preço e Médias Móveis:** normalizados pela divisão do valor inicial (escala relativa).
- **Indicadores como RSI, Volatilidade e Retornos:** normalizados via Z-score.
- **Volume:** transformado em escala logarítmica (`log1p`) para mitigar distorções.

Essa etapa prepara os dados para entrada no agente de RL.


In [13]:
import numpy as np

def normalizar_por_ativo(df):
    df = df.copy()

    # Normalização por divisão (Close e SMAs)
    preco_base = df["Close"].iloc[0]
    df["Close_Norm"] = df["Close"] / preco_base
    df["SMA5_Norm"] = df["SMA5"] / preco_base
    df["SMA20_Norm"] = df["SMA20"] / preco_base

    # Z-score: Return, Volatility, RSI
    for col in ["Return", "Volatility", "RSI"]:
        media = df[col].mean()
        desvio = df[col].std()
        df[f"{col}_Z"] = (df[col] - media) / desvio

    # Log transformação: Volume
    df["Volume_Log"] = np.log1p(df["Volume"])

    return df

# Aplicar normalização a todos os ativos
ativos_normalizados = {}

for nome_ativo, df in ativos_rl.items():
    df_norm = normalizar_por_ativo(df)
    ativos_normalizados[nome_ativo] = df_norm

    print(f"\n✅ Normalização concluída para {nome_ativo}")
    display(df_norm[["Date", "Close_Norm", "SMA5_Norm", "SMA20_Norm", "Return_Z", "Volatility_Z", "RSI_Z", "Volume_Log"]].dropna().head())



✅ Normalização concluída para VALE3


,Date,Close_Norm,SMA5_Norm,SMA20_Norm,Return_Z,Volatility_Z,RSI_Z,Volume_Log
19,2020-06-02,1.205053,1.175592,1.123878,-0.036562,0.927622,0.970129,17.058208
20,2020-06-03,1.210467,1.189217,1.134401,0.188921,0.928359,1.138867,17.268171
21,2020-06-04,1.255583,1.214347,1.145240,1.834040,0.926500,1.373771,17.443541
22,2020-06-05,1.231897,1.221611,1.151737,-0.983338,0.961007,0.660794,17.499355
23,2020-06-08,1.235732,1.227746,1.159689,0.119669,0.923586,0.505439,17.128114



✅ Normalização concluída para PETR4


,Date,Close_Norm,SMA5_Norm,SMA20_Norm,Return_Z,Volatility_Z,RSI_Z,Volume_Log
19,2020-06-02,1.238426,1.177893,1.090654,2.217101,0.444784,1.486539,18.162365
20,2020-06-03,1.242477,1.195718,1.102778,0.078347,0.364266,1.641829,18.272409
21,2020-06-04,1.240162,1.214931,1.114323,-0.144131,0.336891,1.810238,18.492084
22,2020-06-05,1.278935,1.235301,1.124797,1.291280,-0.031482,1.698693,18.552916
23,2020-06-08,1.303819,1.260764,1.137471,0.779667,-0.249703,1.747114,18.227635



✅ Normalização concluída para BRFS3


,Date,Close_Norm,SMA5_Norm,SMA20_Norm,Return_Z,Volatility_Z,RSI_Z,Volume_Log
19,2020-06-02,1.275259,1.273730,1.190278,-0.466638,-0.569690,0.272085,16.062181
20,2020-06-03,1.237029,1.267176,1.202130,-1.008855,-0.394684,-0.148732,16.533583
21,2020-06-04,1.280721,1.269798,1.215893,1.142287,-0.436542,0.358184,15.952520
22,2020-06-05,1.239760,1.265101,1.226707,-1.074886,-0.164485,0.439290,16.338351
23,2020-06-08,1.269798,1.260513,1.233260,0.776911,-0.109478,0.538565,16.043092


## 🧠 Etapa 5.3 — Montagem do Espaço de Estados

O espaço de estados define quais variáveis serão observadas pelo agente em cada timestep.

Para o MVP, o vetor de estado será composto pelas variáveis técnicas normalizadas:
- `Close_Norm`
- `SMA5_Norm`
- `SMA20_Norm`
- `Return_Z`
- `Volatility_Z`
- `RSI_Z`
- `Volume_Log`

Esse vetor será calculado para cada ativo, descartando as linhas com valores ausentes (devido aos períodos iniciais dos indicadores).


In [14]:
# Colunas que compõem o vetor de estado
colunas_estado = [
    "Date",
    "Close_Norm",
    "SMA5_Norm",
    "SMA20_Norm",
    "Return_Z",
    "Volatility_Z",
    "RSI_Z",
    "Volume_Log"
]

# Construção do dataset final de estados
ativos_estado = {}

for nome_ativo, df in ativos_normalizados.items():
    df_estado = df[colunas_estado].dropna().reset_index(drop=True)
    ativos_estado[nome_ativo] = df_estado

    print(f"\n🧠 Vetor de estados final para {nome_ativo}")
    display(df_estado.head())



🧠 Vetor de estados final para VALE3


,Date,Close_Norm,SMA5_Norm,SMA20_Norm,Return_Z,Volatility_Z,RSI_Z,Volume_Log
0,2020-06-02,1.205053,1.175592,1.123878,-0.036562,0.927622,0.970129,17.058208
1,2020-06-03,1.210467,1.189217,1.134401,0.188921,0.928359,1.138867,17.268171
2,2020-06-04,1.255583,1.214347,1.145240,1.834040,0.926500,1.373771,17.443541
3,2020-06-05,1.231897,1.221611,1.151737,-0.983338,0.961007,0.660794,17.499355
4,2020-06-08,1.235732,1.227746,1.159689,0.119669,0.923586,0.505439,17.128114



🧠 Vetor de estados final para PETR4


,Date,Close_Norm,SMA5_Norm,SMA20_Norm,Return_Z,Volatility_Z,RSI_Z,Volume_Log
0,2020-06-02,1.238426,1.177893,1.090654,2.217101,0.444784,1.486539,18.162365
1,2020-06-03,1.242477,1.195718,1.102778,0.078347,0.364266,1.641829,18.272409
2,2020-06-04,1.240162,1.214931,1.114323,-0.144131,0.336891,1.810238,18.492084
3,2020-06-05,1.278935,1.235301,1.124797,1.291280,-0.031482,1.698693,18.552916
4,2020-06-08,1.303819,1.260764,1.137471,0.779667,-0.249703,1.747114,18.227635



🧠 Vetor de estados final para BRFS3


,Date,Close_Norm,SMA5_Norm,SMA20_Norm,Return_Z,Volatility_Z,RSI_Z,Volume_Log
0,2020-06-02,1.275259,1.273730,1.190278,-0.466638,-0.569690,0.272085,16.062181
1,2020-06-03,1.237029,1.267176,1.202130,-1.008855,-0.394684,-0.148732,16.533583
2,2020-06-04,1.280721,1.269798,1.215893,1.142287,-0.436542,0.358184,15.952520
3,2020-06-05,1.239760,1.265101,1.226707,-1.074886,-0.164485,0.439290,16.338351
4,2020-06-08,1.269798,1.260513,1.233260,0.776911,-0.109478,0.538565,16.043092


## 💾 Salvando os Vetores de Estado Final

Os vetores de estado, já normalizados e sem valores ausentes, serão salvos em arquivos `.csv` nomeados no padrão `<ativo>_estado.csv`.

Esses arquivos estarão localizados no diretório `../dados`, ao lado dos arquivos originais e normalizados.


In [15]:
# Diretório de saída (mesmo onde estavam os dados brutos)
diretorio_saida = "../dados"

# Salvar os vetores de estado
for nome_ativo, df_estado in ativos_estado.items():
    nome_arquivo_saida = f"{nome_ativo}_estado.csv"
    caminho_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

    df_estado.to_csv(caminho_saida, index=False)
    print(f"✅ Vetor de estado salvo: {nome_arquivo_saida}")


✅ Vetor de estado salvo: VALE3_estado.csv
✅ Vetor de estado salvo: PETR4_estado.csv
✅ Vetor de estado salvo: BRFS3_estado.csv


## 💼 Construção do Vetor de Portfólio

Para simular o estado interno do agente a cada passo de tempo, será criado um vetor de portfólio contendo:

- As posições binárias em cada ativo (`0` = fora do ativo, `1` = comprado);
- O saldo de caixa normalizado (`1.0` representa o total inicial, equivalente a R$ 100.000,00).

Esse vetor será sincronizado com a linha temporal utilizada nos vetores de estado dos ativos e servirá como base para o ambiente de simulação de decisões futuras.


In [16]:
import pandas as pd
import os

# Caminho onde os vetores de estado foram salvos
caminho_dados = "../dados"
arquivo_estado_base = "VALE3_estado.csv"  # Usaremos VALE3 como referência temporal

# Leitura da linha temporal base
df_base = pd.read_csv(os.path.join(caminho_dados, arquivo_estado_base))
datas = df_base["Date"]

# Construção do vetor de portfólio
df_portfolio = pd.DataFrame({
    "Date": datas,
    "Position_VALE3": 0,
    "Position_PETR4": 0,
    "Position_BRFS3": 0,
    "Cash_Norm": 1.0
})

# Salvamento do vetor
caminho_saida = os.path.join(caminho_dados, "vetor_portfolio.csv")
df_portfolio.to_csv(caminho_saida, index=False)

print(f"✅ Vetor de portfólio salvo com sucesso em: {caminho_saida}")
display(df_portfolio.head())


✅ Vetor de portfólio salvo com sucesso em: ../dados/vetor_portfolio.csv


,Date,Position_VALE3,Position_PETR4,Position_BRFS3,Cash_Norm
0,2020-06-02,0,0,0,1.0
1,2020-06-03,0,0,0,1.0
2,2020-06-04,0,0,0,1.0
3,2020-06-05,0,0,0,1.0
4,2020-06-08,0,0,0,1.0


## 🧭 Divisão Temporal dos Dados (Treinamento, Validação e Teste)

Devido à natureza sequencial dos dados financeiros, a divisão dos conjuntos não pode ser feita de forma aleatória.

Será adotada uma separação baseada em proporções cronológicas:
- Treinamento: primeiros 70%
- Validação: 15% seguintes
- Teste: últimos 15%

Essa divisão será aplicada apenas aos **vetores de estado dos ativos**, mantendo o vetor de portfólio inalterado.  
Durante a simulação, o portfólio será atualizado passo a passo em tempo real, a partir do seu estado inicial.


In [18]:
import math

# Diretório onde os arquivos estão
caminho_dados = "../dados"

# Arquivos a particionar (vetores de estado dos ativos)
arquivos_estado = [
    "VALE3_estado.csv",
    "PETR4_estado.csv",
    "BRFS3_estado.csv"
]

# Aplicar a divisão temporal aos vetores de estado
for nome_arquivo in arquivos_estado:
    caminho_arquivo = os.path.join(caminho_dados, nome_arquivo)
    df = pd.read_csv(caminho_arquivo)

    n_total = len(df)
    n_train = math.floor(n_total * 0.7)
    n_val = math.floor(n_total * 0.15)
    n_test = n_total - n_train - n_val  # garante 100%

    # Slices dos conjuntos
    df_train = df.iloc[:n_train]
    df_val = df.iloc[n_train:n_train+n_val]
    df_test = df.iloc[n_train+n_val:]

    # Salvar conjuntos
    base_nome = nome_arquivo.replace(".csv", "")
    df_train.to_csv(os.path.join(caminho_dados, f"{base_nome}_train.csv"), index=False)
    df_val.to_csv(os.path.join(caminho_dados, f"{base_nome}_val.csv"), index=False)
    df_test.to_csv(os.path.join(caminho_dados, f"{base_nome}_test.csv"), index=False)

    print(f"✅ Divisão aplicada a {nome_arquivo}:")
    print(f"   Treinamento: {len(df_train)} linhas")
    print(f"   Validação:  {len(df_val)} linhas")
    print(f"   Teste:       {len(df_test)} linhas")

# NOTA: vetor_portfolio.csv NÃO é particionado, pois será atualizado dinamicamente no ambiente.


✅ Divisão aplicada a VALE3_estado.csv:
   Treinamento: 857 linhas
   Validação:  183 linhas
   Teste:       185 linhas
✅ Divisão aplicada a PETR4_estado.csv:
   Treinamento: 857 linhas
   Validação:  183 linhas
   Teste:       185 linhas
✅ Divisão aplicada a BRFS3_estado.csv:
   Treinamento: 857 linhas
   Validação:  183 linhas
   Teste:       185 linhas
